# Band
The notebook created after the *band* module.

Demonstrates domain classes, methods and functions related to the concept of musical band, as well as the mandatory `__init__()`, `__str__()` and `__eq__()` methods. The `__eq__()` method notes the problem of comparing unhashable objects.

Introduces static fields/attributes and static methods (`@staticmethod`).

Illustrates iterators and generators.

Shows how to serialize `Band` objects to JSON.

## Setup / Data

In [ ]:
from datetime import date, datetime, time
import json

from music.musician import Musician
from util.utility import format_date

from testdata.musicians import *

## The *Band* class
The class describing the concept of musical band. It is assumed that a musician is sufficiently described by their name and whether they are a solo musician or a member of a band. It includes a list of `Musician` objects (band members) and the dates when the band started/stopped performing together.

Introduces `@staticmethod`s, iterators (the `__iter__()` and `__next__()` methods) and generators. Implicitly introduces exception handling as well (in demonstrating iterators and generators).

In [ ]:
class Band():
    """The class describing the concept of a music group/band.
    It includes a list of Musician objects (band members)
    and the dates when the band started/stopped performing together.
    """

    # Class variables: like static fields in Java; typically defined and initialized before __init__()
    # Insert a class variable (static field), such as genres, date_pattern,...

    genres = ['rock', 'blues', 'pop', 'alternative', 'unknown']

    def __init__(self, name, *members, start=date.today(), end=date.today()):
        # pass                                            # introduce and initialize iterator counter, self.__i
        self.name = name
        self.members = members
        self.start = start
        self.end = end
        self.__i = 0

    def __str__(self):
        n = self.name
        m = f'({", ".join([member.name for member in self.members]) if self.members else "members unknown"})'
        s = self.start.year
        e = self.end.year
        return f'{n} {m}, {s}-{e}'

    def __eq__(self, other):
        # Musician objects are unhashable, so comparing the members tuples from self and other directly does not work;
        # see https://stackoverflow.com/a/14721133/1899061, https://stackoverflow.com/a/17236824/1899061
        # return self == other if isinstance(other, Band) else False    # No! Musician objects are unhashable!

        t = isinstance(other, Band)
        n = self.name == other.name

        # # members must be compared 'both ways', because the two tuples can be of different length
        # m_diff_1 = [x for x in self.members if x not in other.members]
        # m_diff_2 = [x for x in other.members if x not in self.members]
        # m = len(m_diff_1) == len(m_diff_2) == 0

        # members must be compared 'both ways', because the two tuples can be of different length
        m = all([x in self.members for x in other.members]) and all([x in other.members for x in self.members])

        y = (self.start == other.start) and (self.end == other.end)
        return t and n and m and y

    @staticmethod
    def is_date_valid(d):
        """It is assumed that a band does not perform together since more than ~60 years ago.
        So, the valid date to denote the start of a band's career is between Jan 01, 1960, and today.
        """

        return date(1954, 7, 5) <= d <= date.today() if isinstance(d, date) else False

    def __iter__(self):
        """Once __iter__() and __next__() are implemented in a class,
        we can create an iterator object by calling the iter() built-in function on an object of the class,
        and then call the next() built-in function on that object.
        It is often sufficient to just return self in __iter__(),
        if the iterator counter such as self.__i is introduced and initialized in __init__().
        Alternatively, the iterator counter (self.__i) is introduced and initialized here.
        """

        self.__i = 0
        return self               # sufficient if the iterator counter is introduced and initialized in __init__()

    def __next__(self):
        if self.__i < len(self.members):
            next_m = self.members[self.__i]
            self.__i += 1
            return next_m
        else:
            raise StopIteration

#### The basic methods
Check the basic methods (`__init__()`, `__str__()`,...) of the `Band` class.

In [ ]:
theBeatles = Band('The Beatles', *[johnLennon, paulMcCartney, georgeHarrison, ringoStarr, ],
                  start=date(1957, 7, 6), end=date(1970, 4, 10))
print(theBeatles)
beatles = Band('The Beatles', *[johnLennon, georgeHarrison, paulMcCartney, ringoStarr, Musician('Pete Best')],
               start=date(1957, 7, 6), end=date(1970, 4, 10))
print(theBeatles == beatles)

#### Class variables
Python class variables are much like `static` fields in Java. They are typically defined and initialized before `__init__()`.

In [ ]:
print(Band.genres)

#### Class methods
Check the date validator (the `@staticmethod` `is_date_valid(<date>)`).

In [ ]:
print(Band.is_date_valid(date(1957, 7, 6)))

#### Iterators
Check the iterator defined in the `Band` class. After the iterator is exhausted, repeated attempts to run the iterator fail before the iterator is re-initialized.

In [ ]:
for _ in range(len(theBeatles.members)):
    print(theBeatles.__next__().name)
# print(theBeatles.__next__().name)             # iterator exhausted, must be re-initiated
print()

# i = iter(theBeatles)
# print(i)
i = iter(theBeatles)                            # re-initiating the iterator
for _ in range(len(theBeatles.members)):
    print(next(theBeatles))
print()

# # Alternatively
# theBeatles.__iter__()
# for _ in range(len(theBeatles.members)):
#     print(theBeatles.__next__().name)
# print()

# Repeated attempt to run the iterator fails, because the iterator is exhausted

#### Generators
Develop a simple generator to demonstrate generator objects and the `yield` command.

In [ ]:
def next_member(band):
    """Generator that shows members of a band, one at a time.
    yield produces a generator object, on which we call the next() built-in function.
    A great tutorial on generators: https://realpython.com/introduction-to-python-generators/.
    """

    for member in band.members:
        input('Next: ')
        yield member
        print('Yeah!')

In [ ]:
member_generator = next_member(theBeatles)
print(member_generator)
print(type(member_generator))
print()

while True:
    try:
        print(next(member_generator))
    except:
        break

#### Generator expressions

In [ ]:
print(i**2 for i in range(4))
print(next((i**2 for i in range(4))))
print(next((i**2 for i in range(4))))
ge = (i**2 for i in range(4))
print(ge)
print(next(ge))     # 0
print(next(ge))     # 1
print(next(ge))     # 4
print(next(ge))     # 9
print(next(ge))     # raises StopIteration

## JSON

In [ ]:
class BandEncoder(json.JSONEncoder):
    """JSON encoder for Band objects (cls= parameter in json.dumps()).
    """

    def default(self, band):
        # recommendation: always use double quotes with JSON

        pass


def band_py_to_json(band):
    """JSON encoder for Band objects (default= parameter in json.dumps()).
    """


def band_json_to_py(band_json):
    """JSON decoder for Band objects (object_hook= parameter in json.loads()).
    """

#
if __name__ == "__main__":

    from testdata.musicians import *

    # Demonstrate JSON encoding/decoding of Band objects
    # Single object
    print()

    # List of objects
    print()


